In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import numpy as np                   # Numerical computation library
import pandas as pa                  # Dataframe and Data manipulation Library
import matplotlib.pyplot as plt      # Basic plotting functionality Library
import seaborn as sn                 # Advanced Data visualization Library
import numpy.random as rn
import matplotlib.cm as cm
from sklearn.linear_model import LinearRegression
import matplotlib.colors as pltco # A package to help with coloring plots
from sklearn import preprocessing

[x for x in dir(list) if not '__' in x];

In [24]:
# Current Workforce demographic for the state of colorado input from dropbox
workforce_demographic2018 = pa.read_csv('https://www.dropbox.com/s/w1zzumsadlgg9pn/Colorado_Current_Workforce_Demographic.csv?dl=1')


workforce_demographic2018.head()

,AREA,ST,STATE,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC_Q,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,8,CO,Colorado,00-0000,All Occupations,total,"2,620,640",0.3,1000.000,1.00,...,20.34,32.49,50.05,"22,480","28,110","42,310","67,570","104,110",NaN,NaN
1,8,CO,Colorado,11-0000,Management Occupations,major,"116,770",1.1,44.556,0.85,...,56.15,78.38,#,"55,700","79,630","116,800","163,030",#,NaN,NaN
2,8,CO,Colorado,11-1011,Chief Executives,detailed,570,5.3,0.216,0.16,...,86.63,#,#,"85,110","118,210","180,190",#,#,NaN,NaN
3,8,CO,Colorado,11-1021,General and Operations Managers,detailed,"39,200",1.9,14.958,0.95,...,51.51,81.71,#,"46,310","67,650","107,140","169,970",#,NaN,NaN
4,8,CO,Colorado,11-1031,Legislators,detailed,630,8.0,0.239,0.69,...,*,*,*,"21,230","23,970","48,260","68,270","100,430",True,NaN


In [25]:
workforce_demographic2018.dtypes

AREA          int64
ST           object
STATE        object
OCC_CODE     object
OCC_TITLE    object
OCC_GROUP    object
TOT_EMP      object
EMP_PRSE     object
JOBS_1000    object
LOC_Q        object
H_MEAN       object
A_MEAN       object
MEAN_PRSE    object
H_PCT10      object
H_PCT25      object
H_MEDIAN     object
H_PCT75      object
H_PCT90      object
A_PCT10      object
A_PCT25      object
A_MEDIAN     object
A_PCT75      object
A_PCT90      object
ANNUAL       object
HOURLY       object
dtype: object

In [26]:
def remove_strings (X):
    if '-' in X:
        K = X.find('-')
        return X[:K]
    if '*' in X:
        K = X.find('*')
        return np.nan
    if '#' in X:
        K = X.find('#')
        return np.nan
    else:  
        return X

def convertFloat(dataframe, feature):
    for K in dataframe.index: 
        dataframe.loc[K,feature] = remove_strings(dataframe.loc[K, feature])
        dataframe.dropna(subset=[feature], inplace=True)
        
    
    dataframe.loc[:, feature] = dataframe.loc[:,feature].astype("float")  
    return dataframe

In [27]:
workforce_demographic2018 = convertFloat(workforce_demographic2018, "OCC_CODE")  ##this works to convert OCC_Code to float64

In [28]:
workforce_demographic2018['TOT_EMP'] = workforce_demographic2018['TOT_EMP'].str.replace(',','')  
workforce_demographic2018['A_MEAN'] = workforce_demographic2018['A_MEAN'].str.replace(',','')

In [29]:
workforce_demographic2018 = convertFloat(workforce_demographic2018, "TOT_EMP")  ##this works to convert OCC_Code to float64

In [30]:
workforce_demographic2018 = convertFloat(workforce_demographic2018, "H_MEAN")  ##this works to convert Hourly Mean to float64

In [31]:
workforce_demographic2018 = convertFloat(workforce_demographic2018, "A_MEAN")  ##this works to convert Anual Mean to float64

In [32]:
workforce_demographic2018['OCC_GROUP'] = workforce_demographic2018['OCC_GROUP'].astype(str)

In [33]:
major_groups_df = workforce_demographic2018.loc[workforce_demographic2018.OCC_GROUP.apply(lambda cat: 'major' in cat)]
detailed_groups_df = workforce_demographic2018.loc[workforce_demographic2018.OCC_GROUP.apply(lambda cat: 'detailed' in cat)]

In [34]:
# Create x, where x the 'scores' column's values as floats
x = major_groups_df[['TOT_EMP']]
y = major_groups_df[['H_MEAN']]
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)
y_scaled = min_max_scaler.fit_transform(y)

# Run the normalizer on the dataframe
x_normalized = pa.DataFrame(x_scaled)
y_normalized = pa.DataFrame(y_scaled)


In [20]:
def cluster_plot(cluster, X):
       
    n_clusters = cluster.get_params()['n_clusters']
    cmap_bold = pltco.ListedColormap([cm.hot(k/n_clusters) for k in range(n_clusters) ])  
    # We need a color map that expands to match the number of clusters

    plt.scatter(X[:, 0], X[:, 1], c=cluster.fit_predict(X), cmap=cmap_bold,marker='.')

In [22]:
km = KMeans(n_clusters=5).fit(major_groups_df[['TOT_EMP', 'H_MEAN']])

In [86]:
cluster_map = pa.DataFrame()
cluster_map['Major Group'] = major_groups_df.OCC_TITLE.values
cluster_map['cluster'] = km.labels_
cluster_map["Total Employees"] = major_groups_df.TOT_EMP.values
cluster_map["Hourly Mean"] = major_groups_df.H_MEAN.values

In [35]:
total_employees_anuanl_mean_major = major_groups_df[['TOT_EMP', 'H_MEAN']].to_numpy()
normalized_variables = [x_normalized, y_normalized]
#cluster_plot(km, total_employees_anuanl_mean_major)

In [39]:
d = {'TOT_EMP': [x_normalized], 'H_MEAN': [y_normalized]}
df = pa.DataFrame(d)
#df[[x_normalized, y_normalized]].to_numpy()
df.head()

,TOT_EMP,H_MEAN
0,0 0 0.309646 1 0.517243 2 0.3...,0 0 1.000000 1 0.492832 2 0.6...


In [88]:
cluster_map[cluster_map.cluster == 1]

,Major Group,cluster,Total Employees,Hourly Mean
12,Food Preparation and Serving Related Occupations,1,255250.0,13.12
15,Sales and Related Occupations,1,291900.0,23.01


In [89]:
cluster_map[cluster_map.cluster == 2]

,Major Group,cluster,Total Employees,Hourly Mean
1,Business and Financial Operations Occupations,2,190330.0,37.87
7,"Education, Training, and Library Occupations",2,143090.0,26.40
9,Healthcare Practitioners and Technical Occupat...,2,146200.0,40.35
18,Construction and Extraction Occupations,2,140690.0,24.03
21,Transportation and Material Moving Occupations,2,150880.0,20.34


In [90]:
cluster_map[cluster_map.cluster == 3]

,Major Group,cluster,Total Employees,Hourly Mean
16,Office and Administrative Support Occupations,3,361390.0,19.68


In [91]:
cluster_map[cluster_map.cluster == 4]

,Major Group,cluster,Total Employees,Hourly Mean
0,Management Occupations,4,116770.0,63.34
2,Computer and Mathematical Occupations,4,116930.0,46.58
13,Building and Grounds Cleaning and Maintenance ...,4,83120.0,15.01
14,Personal Care and Service Occupations,4,93270.0,15.01
19,"Installation, Maintenance, and Repair Occupations",4,101450.0,24.75
20,Production Occupations,4,102540.0,19.71


In [92]:
cluster_map[cluster_map.cluster == 0]

,Major Group,cluster,Total Employees,Hourly Mean
3,Architecture and Engineering Occupations,0,61970.0,43.80
4,"Life, Physical, and Social Science Occupations",0,30330.0,36.62
5,Community and Social Service Occupations,0,39470.0,23.98
6,Legal Occupations,0,22540.0,54.54
8,"Arts, Design, Entertainment, Sports, and Media...",0,40640.0,26.55
10,Healthcare Support Occupations,0,65740.0,17.07
11,Protective Service Occupations,0,59080.0,24.04
17,"Farming, Fishing, and Forestry Occupations",0,7050.0,15.60
